In [ ]:
import json
import os
import brightway2 as bw
import numpy as np
import pandas as pd

# For plotting
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import colorlover as cl
from IPython.display import HTML

# Load data

In [ ]:
path = '../generated_files/gsa_results/cge_N200_gt_database/sobol_indices.json'
with open(path) as f:
    sa_dict = json.load(f)
parameters = sa_dict['parameters']
n_parameters = len(parameters)

# Extract total index into a dictionary and dataframe
total_dict = {}
total_dict['parameters'] = parameters
for k in sa_dict.keys():
    if k != 'parameters':
        all_vals = sa_dict[k]['ST']
        n_all = len(all_vals)
        total_dict[k] = all_vals[n_all-n_parameters:]
total_df = pd.DataFrame(total_dict)

In [ ]:
names = total_df['parameters'].tolist()

def normalize_df(df):
    norm_df = pd.DataFrame([],index = df.index, columns = df.columns)
    norm_df['parameters'] = df['parameters']
    for col in df.columns:
        if col != 'parameters':
            norm_df[col] = df[col] / sum(df[col])
    return norm_df

In [ ]:
df = normalize_df(total_df)

In [ ]:
if 'cge' in path:
    df = df.set_index('parameters')
    new_index = ['co2_emissions',
                 'gross_power_per_well', 
                 'average_depth_of_wells',
                 'initial_harmonic_decline_rate',
                 'success_rate_primary_wells',
                 'lifetime',
                 'collection_pipelines', 
                 'installed_capacity', 
                 'capacity_factor', 
                 'auxiliary_power',
                 'specific_diesel_consumption', 
                 'specific_steel_consumption',
                 'specific_cement_consumption', 
                 'specific_drilling_mud_consumption',
                 'production_to_injection_ratio',
                 'success_rate_exploration_wells', 
                 'success_rate_makeup_wells']
    df = df.reindex(new_index)
    df['index'] = np.arange(len(df))
    df = df.reset_index()
    df = df.set_index('index')
elif 'ege' in path:
    df = df.sort_values(by=['climate change total'], ascending=False)
    df['index'] = np.arange(len(df))
    df = df.set_index('index')

In [ ]:
my_colors = ['rgb(255,237,0)',
             'rgb(184,255,185)',
             'rgb(30,221,109)',
             'rgb(255,210,255)',
             'rgb(248,114,225)',
             'rgb(137,231,255)',
             ]
colors = cl.scales['11']['qual']['Paired'] + my_colors
HTML(cl.to_html( colors ))

In [ ]:
fig = go.Figure()
ydata = df.columns[1:].tolist()
ydata = [d.capitalize() for d in ydata]
f = [ind for ind,yd in enumerate(ydata) if "Fossils"==yd][0]
ydata[f] = 'Fossil resources'

for i in df.index:
    xdata = df.loc[i][1:].tolist()
    name = df.loc[i][0].replace('_', ' ').capitalize()
    if name.lower() == 'co2 emissions':
        name = 'Direct CO2 emissions'
    elif name.lower() == 'collection pipelines':
        name = 'Collection pipelines length'
    elif 'success' in name.lower():
        name = name[:13] + 'of ' + name[13:]
    elif '0to1' in name.lower():
        name = name[:-4] + '0 ot 1'
    fig.add_bar(name = name, 
                x = xdata,
                y = ydata,
                orientation='h',
                marker_color=colors[i])
    fig.update_xaxes(range = [0,1])
    
    fig.update_layout(barmode='stack',
                      font_size = 18,
                      width = 1130, 
                      height = 500,
                      legend_traceorder  = 'normal',
                      yaxis = dict(autorange="reversed"),
                      font_family = 'Arial',
                      margin = dict(l=0,r=0,t=0,b=0),
                      font_color = 'black',
                      xaxis = dict(tickmode = 'array', 
                                   tickvals = [0, 0.2, 0.4, 0.6, 0.8, 1],
                                   ticktext = [0, 0.2, 0.4, 0.6, 0.8, 1])
                     )
fig

In [ ]:
# Save image
path_images = "write_images"
if not os.path.exists(path_images):
    os.mkdir(path_images)
    
start_end = [i for i, ltr in enumerate(path) if ltr == '/']
start = start_end[-2]
end = start_end[-1]

path_fig = os.path.join(path_images, 'sobol_' + path[start+1:end] + '.png')
fig.write_image(path_fig)